# Use examples of [premise](https://github.com/romainsacchi/premise)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `premise` to create life cycle invneotries databases for prospective environmental impact assessment.

This library extract useful information from IAM model output files (such as those of REMIND or IMAGE) and aligns inventories in the ecoinvent database accordingly.

With version 0.2.0, the following transformation are available:

* `update_electricity()`: create regional electricity markets and adjust efficiency of power plants
* `update_cement()`: creates regional markets for clinker production and adjust clinker production efficiency
* `update_steel()`: creates regional markets for steel and adjust steel production efficiency and the supply of secondary steel
* `update_cars()`: produces fleet average cars and relinks to activities consuming pasenger car trnasport
* `update_trucks()`: produces fleet average trucks and relinks to activities consuming lorry trnasport
* `update_solar_PV()`: updates efficiency of solar PV modules

Additional documentation on the methodology is available [here](https://premise.readthedocs.io/en/latest/introduction.html).

## Requirements

* **Pyhton 3.9 is highly recommended**
* a user license for ecoinvent v.3

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have a open `brightway2` project with abiosphere as well as an
ecoinvent v.3 database registered in that project.

In [ ]:
from premise import *
import brightway2 as bw

In [ ]:
bw.projects.set_current("your_bw_project")

### List of available scenarios

Some scenarios come installed with the library.
They are stored in `data/iam_ouput_files` from the root directory.
They are all within the SSP2 pathway (middle of the road).

With REMIND:
* "SSP2-Base"
* "SSP2-NPi"
* "SSP2-NDC"
* "SSP2-PkBudg1300"
* "SSP2-PkBudg1100"
* "SSP2-PkBudg900"

With IMAGE:
* "SSP2-Base"


### Database creation from default scenarios

To create a scenario using REMIND's SSP2 Base pathway, from ecoinvent 3.6 for the year 2028, one would execute the following cell. This leads to the extraction of the database, some cleanup as well as importing a few additional inventories.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
            source_db="ecoinvent 3.6 cutoff", # <-- name of the database in the BW2 project. Must be a string.
            source_version="3.6", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1". Must be a string.
            key='xxxxxxxxxxxxxxxxxxxxxxxxx' # <-- decryption key
            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
    )

or here with ecoinvent 3.7.1

In [ ]:
ndb = NewDatabase(
        scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2028}
            ],
        source_db="ecoinvent 3.7 cutoff", # <-- this is NEW.
        source_version="3.7.1", # <-- this is NEW
        key='xxxxxxxxxxxxxxxxxxxxxxxxx'
    )

If you want to create multiple databases at once, just populate the `scenarios` list.

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2020, "exclude": ["update_electricity"]},
                {"model":"remind", "pathway":"SSP2-Base", "year":2030},
                {"model":"remind", "pathway":"SSP2-Base", "year":2040},
                {"model":"remind", "pathway":"SSP2-Base", "year":2050},
            ],
            source_db="ecoinvent 3.7 cutoff", # <-- name of the database. Must be a string.
            source_version="3.7.1", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.7.1"
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
)

In [ ]:
ndb.update_all()

When the database is loaded and the additional inventories imported, you can apply a transformation function.
For example here, we adjust the efficiency of the solar PVs to the two scenarios we have loaded.
We go more in details later.

In [ ]:
ndb.update_solar_PV()

And then, we register these two databases back into brightway2.

In [ ]:
ndb.write_db_to_brightway()

### Database creation from non-default scenarios

If you have some specific IAM scenarios (one that is not included in `premise`) you would like to build a database from, you can specify the directory to those.

**Important remark**: your scenario file must begin with "remind_" or "image_". When using a non-default scenario that you provide yourself, you do not have to provide a decryption key.

In [ ]:
ndb = NewDatabase(
    scenarios = [{"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}],        
    source_db="ecoinvent 3.6 cutoff", # <-- name of the database
    source_version=3.6, # <-- version of ecoinvent
                 )

### Adding inventories
Upon the database extraction, you can import some of your Brightway2-compatible inventories like so:

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030},
                {"model":"remind", "pathway":"SSP2-Base", "year":2030},
            ],
            source_db="ecoinvent 3.7 cutoff", 
            source_version="3.7.1",
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
            additional_inventories= [ # <-- this is NEW
                {"filepath": r"filepath\to\excel_file.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
                {"filepath": r"filepath\to\another_excel_file.xlsx", "ecoinvent version": "3.7"}, # <-- this is NEW
            ] # <-- this is NEW
                 )

# Use case with ecospold2

The source database does not have to be from a brightway2 project.
It can be directly extracted from the bunch of ecospold2 files one gets when downloaded from the [ecoinvent website](https://ecoinvent.org).

For this, one needs to specify the argument `source_db = "ecospold"` as well as `source_file_path`, which is the directory leading to the ecospold files.

For example, here we combine the use of a specific (non-default) IAM scenario file with the use of ecospold2 files as data source (ecoinvent 3.5 in this case).

In [ ]:
ndb = NewDatabase(
        scenarios = [
            {"model":"remind", "pathway":"my_special_scenario", "year":2028, "filepath":r"C:\Users\sacchi_r\Downloads\REMIND"}
        ],        
        source_type="ecospold", # <--- this is NEW
        source_file_path=r"C:\Users\sacchi_r\Dropbox\Public\ecoinvent 3.5_cutoff_ecoSpold02\datasets", # <-- this is NEW
        source_version="3.5",
    )

# Transformation functions

These functions modify the extracted database:

* **update_electricity()**: alignment of regional electricity production mixes as well as efficiencies for a number of
  electricity production technologies, including Carbon Capture and Storage technologies.
* **update_cement()**: adjustment of technologies for cement production (dry, semi-dry, wet, with pre-heater or not),
  fuel efficiency of kilns, fuel mix of kilns (including biomass and waste fuels) and clinker-to-cement ratio.
* **update_steel()**: adjustment of process efficiency, fuel mix and share of secondary steel in steel markets.
* **update_solar_PV()**: adjustment of solar PV panels efficiency to the year considered.

They can be applied *separately*, *consecutively* or *altogether* (using instead **.update_all()**).
They will apply to all the scenario-specific databases listed in `scenarios`.

In [ ]:
ndb.update_all()

#### Additional options for database creation with cars and truck inventories

Two additional functions are available:
* **update_cars()**: creates updated inventories for fleet average passenger cars and links back to activities that consume
transport.
* **update_trucks()**: creates updated inventories for fleet average lorry trucks and links back to activities that consume
transport.


By default, inventories for passenger cars and trucks are not added upon the creation of the database.
But you can add them by adding the following keys to the scenario dictionary: `passenger cars` and `trucks`.
This will create inventories for passenger cars and trucks for each region of the IAM model.

For example, here we create car inventories in the first scenario (but only for the region *EUR*), and both cars and trucks inventories in the second scenario (for all regions). Leaving an empty dictionary indicates that we do not apply any filter in terms of regions and that we do not provide a fleet composition file (a default one is then used).

In [1]:
from premise import *
import brightway2 as bw
bw.projects.set_current("article_carculator")

In [2]:
ndb = NewDatabase(
            scenarios=[
                {'model':'remind','pathway':'SSP2-Base','year':'2020', "passenger_cars":{"regions":["EUR"]}, "trucks":{"regions":["EUR"]}},
                #{"model":"image", "pathway":"SSP2-Base", "year":2034},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7", 
)

No fleet composition file is provided for passenger_cars.Fleet average vehicles will be built using default fleet projection.
No fleet composition file is provided for trucks.Fleet average vehicles will be built using default fleet projection.

////////////////////// EXTRACTING SOURCE DATABASE ///////////////////////
Getting activity data


100%|████████████████████████████████| 19128/19128 [00:00<00:00, 117561.29it/s]


Adding exchange data to activities


100%|███████████████████████████████| 621719/621719 [00:44<00:00, 13985.72it/s]


Filling out exchange data


100%|██████████████████████████████████| 19128/19128 [00:04<00:00, 4723.92it/s]


Set missing location of datasets to global scope.
Set missing location of production exchanges to scope of dataset.
Correct missing location of technosphere exchanges.
Remove empty exchanges.

/////////////////// IMPORTING DEFAULT INVENTORIES ////////////////////
Importing necessary inventories...

Done!



In [3]:
ndb.update_trucks()


/////////////////// MEDIUM AND HEAVY DUTY TRUCKS ////////////////////
Regional delivery driving cycle is selected. Vehicles will be designed to achieve a minimal range of 400 km.

Capacity utilization assumed (share of available payload used)
+--------------+------+------+------+------+------+
|              | 3.5t | 7.5t | 18t  | 26t  | 40t  |
+--------------+------+------+------+------+------+
|  BEV, 2010   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2011   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2012   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2013   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2014   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2015   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2016   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2017   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2018   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2019   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  BEV, 2020   | 0.6  | 0.41 | 0.42 | 0.38 | 0.36 |
|  FCEV, 2010  | 0.6  | 0.41

0% [##############################] 100% | ETA: 00:00:00 | Item ID: 1191
Total time elapsed: 00:00:01


The following datasets to import already exist in the source database. They will not be imported
+----------------------------------------------------+--------------------------------+----------------------------+------+
|                        Name                        |       Reference product        |          Location          | File |
+----------------------------------------------------+--------------------------------+----------------------------+------+
|                  Lower retention                   |        Lower retention         |            GLO             |      |
| Gasoline production, from methanol-to-gasoline pro |      Gasoline, synthetic       |            RER             |      |
| Plant oil from crude oil | refining of vegetable o | Plant oil from crude oil | ref |            RER             |      |
|             Ethanol from maize starch              |   Ethanol from maize starch    |            RER             |      |
|  Ethanol, from maize starch, at f

RER  not found in regex


Cannot find the IAM location for RER w/o RU from IAM model remind.


In [ ]:
ndb.update_electricity()

Then we need to explicitly run `update_cars()` and/or `update_trucks()` (or `update_all()`).

In [ ]:
ndb.update_cars()

In [ ]:
ndb.update_trucks()

In [ ]:
ndb.write_db_to_brightway()

You can also give your datababase a custom name.

In [5]:
ndb.write_db_to_brightway(name=["my_custom_name_1", "my_custom_name_2"])

Write new database(s) to Brightway2.
19984 datasets
644054 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:04:37


Title: Writing activities to SQLite3 database:
  Started: 03/16/2021 12:08:25
  Finished: 03/16/2021 12:13:02
  Total time elapsed: 00:04:37
  CPU %: 54.30
  Memory %: 37.41
Created database: my_custom_name_1
20883 datasets
663097 exchanges
0 unlinked exchanges
  


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:33


Title: Writing activities to SQLite3 database:
  Started: 03/16/2021 12:17:30
  Finished: 03/16/2021 12:23:03
  Total time elapsed: 00:05:33
  CPU %: 48.10
  Memory %: 35.99
Created database: my_custom_name_2


### Fleet files

A last word about passenger cars and trucks: it is possible to pass a custom fleet composition file to generate fleet average inventories and limit those to specific regions (here, the European region).

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,
                 "passenger cars": {"regions":["EUR", "NEU"],
                                    "fleet file":r"filepath/to/fleet_file.csv.csv"},
                 "trucks": {"regions":["EUR"]}
                },
                {"model":"image", "pathway":"SSP2-Base", "year":2030,},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7"
    )

### Exclude specific functions
Finally, we can exclude some transformation functions when executing `update_all()` like so:

In [ ]:
ndb = NewDatabase(
            scenarios=[
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,
                 "exclude": ["update_steel"], # <-- do not execute update_seel()
                 "passenger cars": {"regions":["EUR"]},"trucks": {"regions":["EUR"]}
                },
                {"model":"remind", "pathway":"SSP2-Base", "year":2030,},
            ],
            key='xxxxxxxxxxxxxxxxxxxxxxxxx'
            source_db="ecoinvent 3.7 cutoff",
            source_version="3.7", 
)

# Export

### As a Brightway2 database

Export the modified database to brightway2

In [ ]:
ndb.write_db_to_brightway()

### As a sparse matrix representation

Or export it as a sparse matrix representation.

This will export four files:

* "A_matrix.csv": matrix coordinates and values of shape (index of activity; index of product; value) for the technosphere
* "A_matrix_index.csv": labels for indices for A matrix of shape (name of activity, reference product, unit, location, index)
* "B_matrix.csv": matrix coordinates and values of shape (index of activity; index of biosphere flow; value) for the biosphere
* "B_matrix_index.csv": labels for indices for B matrix of shape (name of biosphere flow, main compartment, sub-compartmnet, unit, index)

As a convenience, you can specifiy a directory where to store the exported matrices.
If the directory does not exist, it will be created.
If you leave it unspecified, they will be stored in **data/matrices** in the root folder of the library.

In [ ]:
ndb.write_db_to_matrices(filepath=r"C:/Users/sacchi_r/Downloads/exported_matrices")

### As a SimaPro CSV file

In [ ]:
ndb.write_db_to_simapro(filepath=r"C:/Users/sacchi_r/Downloads/exported_simapro_file")